# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
import requests
from bs4 import BeautifulSoup

def get_book_data(book_url: str) -> dict:
    """
    Извлекает данные о книге с веб-страницы.

    Args:
        book_url (str): URL страницы книги для парсинга

    Returns:
        dict: Словарь с информацией о книге (название, цена, рейтинг, описание и др.)
    """

    # Отправляем запрос к странице
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Извлекаем основные данные
    title = soup.find('h1').text
    price = soup.find('p', class_='price_color').text

    # Находим рейтинг (звезды)
    rating_element = soup.find('p', class_='star-rating')
    rating_classes = rating_element['class']
    rating = [cls for cls in rating_classes if cls != 'star-rating'][0]

    # Описание книги
    description = soup.find('meta', attrs={'name': 'description'})['content'].strip()

    # Таблица с дополнительной информацией
    product_info = {}
    table = soup.find('table', class_='table table-striped')
    for row in table.find_all('tr'):
        header = row.find('th').text
        value = row.find('td').text
        product_info[header] = value

    # Формируем результат
    book_data = {
        'title': title,
        'price': price,
        'rating': rating,
        'description': description,
        **product_info  # Распаковываем всю табличную информацию
    }

    return book_data

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 'Three',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And y

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [18]:
def scrape_books(is_save: bool = False) -> list:
    """
    Собирает данные о всех книгах с сайта Books to Scrape.

    Args:
        is_save (bool): Если True, сохраняет данные в файл 'books_data.txt'

    Returns:
        list: Список словарей с информацией о книгах
    """

    all_books_data = []
    page_number = 1

    while True:
        # Формируем URL страницы
        if page_number == 1:
            page_url = "http://books.toscrape.com/index.html"
            base_url = "http://books.toscrape.com/"
        else:
            page_url = f"http://books.toscrape.com/catalogue/page-{page_number}.html"
            base_url = "http://books.toscrape.com/catalogue/"

        print(f"Парсим страницу {page_number}...")

        try:
            response = requests.get(page_url)

            # Проверяем успешность запроса
            if response.status_code != 200:
                print(f"Страница {page_number} не найдена (статус {response.status_code})")
                break

            soup = BeautifulSoup(response.content, 'html.parser')

            # Находим все карточки книг на странице
            book_cards = soup.find_all('article', class_='product_pod')

            # Если на странице нет книг, выходим
            if not book_cards:
                print("На странице нет книг - достигнут конец каталога")
                break

            # Парсим каждую книгу на странице
            successful_books = 0
            for card in book_cards:
                # Получаем ссылку на страницу книги
                book_relative_link = card.find('h3').find('a')['href']

                # ОБРАБАТЫВАЕМ ССЫЛКИ ДЛЯ ПЕРВОЙ И ОСТАЛЬНЫХ СТРАНИЦ ПО-РАЗНОМУ
                if page_number == 1:
                    # На первой странице ссылки вида: catalogue/a-light-in-the-attic_1000/index.html
                    full_book_url = "http://books.toscrape.com/" + book_relative_link
                else:
                    # На остальных страницах ссылки уже правильные
                    full_book_url = "http://books.toscrape.com/catalogue/" + book_relative_link

                # Получаем данные книги
                try:
                    book_data = get_book_data(full_book_url)
                    if 'error' not in book_data:
                        all_books_data.append(book_data)
                        successful_books += 1
                        print(f"  Собрана книга: {book_data['title']}")
                    else:
                        print(f"  Ошибка: {book_data['error']}")


                except Exception as e:
                    print(f"  Ошибка при парсинге книги: {e}")
                    continue

            print(f"  На странице {page_number} успешно собрано: {successful_books}/{len(book_cards)} книг")

            # Проверяем есть ли следующая страница
            next_button = soup.find('li', class_='next')
            if not next_button:
                print("Достигнута последняя страница каталога")
                break

            # Переходим к следующей странице
            page_number += 1

        except Exception as e:
            print(f"Ошибка при парсинге страницы {page_number}: {e}")
            break

    # Сохранение в файл, если нужно
    if is_save and all_books_data:
        try:
            with open('books_data.txt', 'w', encoding='utf-8') as f:
                for i, book in enumerate(all_books_data, 1):
                    f.write(f"Книга #{i}\n")
                    f.write(f"Название: {book.get('title', 'N/A')}\n")
                    f.write(f"Цена: {book.get('price', 'N/A')}\n")
                    f.write(f"Рейтинг: {book.get('rating', 'N/A')}\n")
                    f.write(f"Описание: {book.get('description', 'N/A')[:100]}...\n")
                    f.write("-" * 50 + "\n\n")
            print(f" Данные сохранены в файл 'books_data.txt'")
        except Exception as e:
            print(f" Ошибка при сохранении файла: {e}")

    print(f"ИТОГО собрано книг: {len(all_books_data)}")
    return all_books_data

In [19]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Парсим страницу 1...
  Собрана книга: A Light in the Attic
  Собрана книга: Tipping the Velvet
  Собрана книга: Soumission
  Собрана книга: Sharp Objects
  Собрана книга: Sapiens: A Brief History of Humankind
  Собрана книга: The Requiem Red
  Собрана книга: The Dirty Little Secrets of Getting Your Dream Job
  Собрана книга: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
  Собрана книга: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
  Собрана книга: The Black Maria
  Собрана книга: Starving Hearts (Triangular Trade Trilogy, #1)
  Собрана книга: Shakespeare's Sonnets
  Собрана книга: Set Me Free
  Собрана книга: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
  Собрана книга: Rip it Up and Start Again
  Собрана книга: Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
  Собрана книга: Olio
  Собрана книга: Mesaerion: The Best Science Fiction Stories 1800-1849

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [22]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
from datetime import datetime

def scheduled_scraping():
    """
    Функция для автоматического запуска парсинга по расписанию.
    Выполняет сбор данных и сохраняет их с timestamp в названии файла.
    """
    print(f" [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Запуск автоматического парсинга...")

    try:
        # Запускаем парсинг
        books_data = scrape_books(is_save=False)  # Не сохраняем в обычный файл

        if books_data:
            # Создаем файл с timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"books_data_{timestamp}.txt"

            # Сохраняем данные
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(f"Автоматический сбор данных от {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Всего книг: {len(books_data)}\n")
                f.write("=" * 60 + "\n\n")

                for i, book in enumerate(books_data, 1):
                    f.write(f"Книга #{i}\n")
                    f.write(f"Название: {book.get('title', 'N/A')}\n")
                    f.write(f"Цена: {book.get('price', 'N/A')}\n")
                    f.write(f"Рейтинг: {book.get('rating', 'N/A')}\n")
                    f.write(f"Описание: {book.get('description', 'N/A')[:100]}...\n")
                    f.write("-" * 50 + "\n\n")

            print(f" Данные сохранены в файл: {filename}")
            print(f" Собрано книг: {len(books_data)}")
        else:
            print(" Не удалось собрать данные")

    except Exception as e:
        print(f" Ошибка при автоматическом парсинге: {e}")

    print(f" [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Парсинг завершен\n")

def setup_scheduler():
    """
    Настраивает расписание для автоматического запуска.
    """
    # Запланировать запуск каждый день в 19:00
    schedule.every().day.at("19:00").do(scheduled_scraping)

    # Дополнительно: тестовый запуск каждые 2 минуты для демонстрации
    schedule.every(2).minutes.do(scheduled_scraping)  # УДАЛИ ЭТУ СТРОКУ В ПРОДАКШЕНЕ

    print(" Планировщик запущен!")
    print(" Расписание:")
    print("   - Ежедневно в 19:00")
    print("   - Каждые 2 минуты (для тестирования)")
    print("   Для остановки: Ctrl+C\n")

def run_scheduler():
    """
    Запускает бесконечный цикл для проверки расписания.
    """
    setup_scheduler()

    try:
        while True:
            schedule.run_pending()
            # Проверяем расписание каждые 30 секунд (вместо постоянной проверки)
            time.sleep(30)

    except KeyboardInterrupt:
        print("\n Планировщик остановлен пользователем")

# Тест 1: Запуск планировщика на короткое время
print(" Запуск теста планировщика на 1 минуту...")

# Настраиваем тестовое расписание (каждую минуту)
schedule.clear()  # Очищаем предыдущие задания
schedule.every(1).minutes.do(scheduled_scraping)  # Каждую минуту для теста

print(" Тестовое расписание установлено: каждую минуту")
print(" Ожидайте запуска... (1 минута)\n")

# Запускаем на 70 секунд (чтобы успеть выполниться 1 раз)
start_time = time.time()
while time.time() - start_time < 70:  # 70 секунд
    schedule.run_pending()
    time.sleep(10)  # Проверяем каждые 10 секунд

print("\n Тест завершен!")
schedule.clear()  # Очищаем расписание после теста
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

 Запуск теста планировщика на 1 минуту...
 Тестовое расписание установлено: каждую минуту
 Ожидайте запуска... (1 минута)

 [2025-11-09 18:59:08] Запуск автоматического парсинга...
Парсим страницу 1...
  Собрана книга: A Light in the Attic
  Собрана книга: Tipping the Velvet
  Собрана книга: Soumission
  Собрана книга: Sharp Objects
  Собрана книга: Sapiens: A Brief History of Humankind
  Собрана книга: The Requiem Red
  Собрана книга: The Dirty Little Secrets of Getting Your Dream Job
  Собрана книга: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
  Собрана книга: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
  Собрана книга: The Black Maria
  Собрана книга: Starving Hearts (Triangular Trade Trilogy, #1)
  Собрана книга: Shakespeare's Sonnets
  Собрана книга: Set Me Free
  Собрана книга: Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
  Собрана книга: Rip it Up and Start Again
  Собрана к

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [30]:
#Создаем папку tests （работаю в google colab)
import os
os.makedirs('tests', exist_ok=True)
print("Папка создана")

Папка создана


In [38]:
# Создаем ОДИН файл tests/test_scraper.py со ВСЕМ кодом внутри
test_code = '''
import pytest
import requests
from bs4 import BeautifulSoup

def get_book_data(book_url: str) -> dict:
    """
    Извлекает данные о книге с веб-страницы.
    """
    try:
        response = requests.get(book_url)
        if response.status_code != 200:
            return {"error": "Страница не доступна"}

        soup = BeautifulSoup(response.content, 'html.parser')

        title_element = soup.find('h1')
        title = title_element.text if title_element else "Название не найдено"

        price_element = soup.find('p', class_='price_color')
        price = price_element.text if price_element else "Цена не найдена"

        rating_element = soup.find('p', class_='star-rating')
        rating = "Рейтинг не найден"
        if rating_element and 'class' in rating_element.attrs:
            rating_classes = rating_element['class']
            rating_words = [cls for cls in rating_classes if cls != 'star-rating']
            rating = rating_words[0] if rating_words else "Рейтинг не найден"

        description_element = soup.find('meta', attrs={'name': 'description'})
        description = description_element['content'].strip() if description_element else "Описание не найдено"

        book_data = {
            'title': title,
            'price': price,
            'rating': rating,
            'description': description
        }

        return book_data

    except Exception as e:
        return {"error": f"Ошибка при парсинге: {str(e)}"}

def test_returns_dict():
    url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    result = get_book_data(url)
    assert isinstance(result, dict), "Функция должна возвращать словарь"
    print("Тест 1 пройден")

def test_has_required_fields():
    url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    result = get_book_data(url)
    assert "title" in result, "Должен быть ключ 'title'"
    assert "price" in result, "Должен быть ключ 'price'"
    assert "rating" in result, "Должен быть ключ 'rating'"
    print("Тест 2 пройден")

def test_title_not_empty():
    url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    result = get_book_data(url)
    assert "title" in result, "Должен быть ключ 'title'"
    assert result["title"] != "", "Название не должно быть пустым"
    print("Тест 3 пройден")
'''

# Создаем файл тестов
with open('tests/test_scraper.py', 'w', encoding='utf-8') as f:
    f.write(test_code)

In [39]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
!python -m pytest tests/test_scraper.py -v

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-4.11.0, typeguard-4.4.4, langsmith-0.4.40
collected 3 items                                                              

tests/test_scraper.py::test_returns_dict PASSED                          [ 33%]
tests/test_scraper.py::test_has_required_fields PASSED                   [ 66%]
tests/test_scraper.py::test_title_not_empty PASSED                       [100%]

============================== 3 passed in 0.55s ===============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```

In [40]:
# 1. Создаем requirements.txt
requirements_content = '''requests==2.31.0
beautifulsoup4==4.12.2
schedule==1.2.0
pytest==7.4.0
'''

with open('requirements.txt', 'w', encoding='utf-8') as f:
    f.write(requirements_content)

In [43]:
# 2. Создаем README.md
readme_content = '''
# Books Scraper Project

## О проекте
Система для сбора данных о книгах с сайта Books to Scrape.

## Что умеет
- Парсит данные о книгах (название, цена, рейтинг)
- Автоматически собирает данные
- Сохраняет результаты в файлы
- Имеет автотесты

## Как запустить
bash
pip install -r requirements.txt
python scraper.py
pytest tests/
'''

with open('README.md', 'w', encoding='utf-8') as f:
  f.write(readme_content)

In [44]:
# 3. Создаем .gitignore
gitignore_content = '''__pycache__/
*.pyc
.DS_Store
'''

with open('.gitignore', 'w', encoding='utf-8') as f:
    f.write(gitignore_content)


In [45]:
# 4. Создаем папку artifacts и пример данных
import os
os.makedirs('artifacts', exist_ok=True)

if os.path.exists('books_data.txt'):

    # Копируем его в папку artifacts
    import shutil
    shutil.copy('books_data.txt', 'artifacts/books_data.txt')
    print("Файл скопирован в artifacts/books_data.txt")

    print("Содержимое файла (первые 5 строк):")
    with open('artifacts/books_data.txt', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 5:  # Показываем только первые 5 строк
                print(f"   {line.strip()}")
            else:
                break
else:
    print("Файл books_data.txt не найден в Colab")

Файл скопирован в artifacts/books_data.txt
Содержимое файла (первые 5 строк):
   Книга #1
   Название: A Light in the Attic
   Цена: £51.77
   Рейтинг: Three
   Описание: It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and...


In [46]:
# Создаем архив с файлами
!zip -r books_scraper_project.zip . -x "*.git*" "*.ipynb_checkpoints*" "*.zip"

print("Архив books_scraper_project.zip создан!")

  adding: .config/ (stored 0%)
  adding: .config/configurations/ (stored 0%)
  adding: .config/configurations/config_default (deflated 15%)
  adding: .config/active_config (stored 0%)
  adding: .config/default_configs.db (deflated 98%)
  adding: .config/config_sentinel (stored 0%)
  adding: .config/.last_survey_prompt.yaml (stored 0%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2025.11.05/ (stored 0%)
  adding: .config/logs/2025.11.05/14.33.36.385956.log (deflated 58%)
  adding: .config/logs/2025.11.05/14.33.44.287731.log (deflated 86%)
  adding: .config/logs/2025.11.05/14.33.54.129583.log (deflated 56%)
  adding: .config/logs/2025.11.05/14.33.45.559498.log (deflated 58%)
  adding: .config/logs/2025.11.05/14.33.53.434728.log (deflated 57%)
  adding: .config/logs/2025.11.05/14.33.13.470069.log (deflated 93%)
  adding: .config/.last_update_check.json (deflated 23%)
  adding: .config/.last_opt_in_prompt.yaml (stored 0%)
  adding: .config/gce (stored 0%)
  adding: .config/hi

In [47]:
# Скачиваем архив на компьютер
from google.colab import files
files.download('books_scraper_project.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>